# Melbourne Urban Heat Model
This notebook predicts surface temperatures across Melbourne suburbs based on NDVI, impervious surface %, building density, and land use.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
# Load the dataset
df = pd.read_csv("melbourne_heat_model.csv")
df["geometry"] = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
df.head()

In [ ]:
# Train model
features = ["ndvi", "impervious_surface_pct", "building_density", "land_use_type"]
X = df[features]
y = df["surface_temp_C"]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)
df["predicted_temp_C"] = model.predict(X)

In [ ]:
# Plot map
fig, ax = plt.subplots(figsize=(10, 8))
gdf["predicted_temp_C"] = df["predicted_temp_C"]
gdf.plot(column="predicted_temp_C", ax=ax, legend=True, cmap="inferno", edgecolor="black", markersize=100)

for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf["Suburb"]):
    ax.text(x + 0.005, y, label, fontsize=9)

plt.title("Predicted Urban Surface Temperature by Suburb – Melbourne")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.tight_layout()
plt.show()